In [2]:
#All the required imports
import requests
import json
from pymongo import MongoClient
import pandas as pd

In [ ]:
# baseurl = "https://data.cityofnewyork.us/resource/qgea-i56i.json?$$app_token=wK7SaysF0k1J53sWS6fKyaxT0"
# year based on input 
# year_url = "&$where=date_extract_y(cmplnt_fr_dt) in (year)"
# month based on input
# month_url = ""
# count_url = "&$select=count(cmplnt_num)"

In [3]:
#Fetch count of existing records from API.
try:
    response = requests.get("https://data.cityofnewyork.us/resource/qgea-i56i.json?$$app_token=wK7SaysF0k1J53sWS6fKyaxT0&$where=date_extract_y(cmplnt_fr_dt) in (2018)&$select=count(cmplnt_num)")
    complaint_count = json.loads(response.text)
    total_count = complaint_count[0]['count_cmplnt_num']
except:
    print(response.status_code)
    print("Error in fetching data from API. Data not found.")

#Total number of records retrived
print(total_count)

#Number of pages to be retriveved.
number_of_pages = int(int(total_count)/50000)

#Number of required records on last page
last_page_number = number_of_pages + 1
last_page_record_count = int(total_count) % 50000
print(last_page_record_count)

#Iterating throgh multiple pages and retrieving data
url = "https://data.cityofnewyork.us/resource/qgea-i56i.json?$$app_token=wK7SaysF0k1J53sWS6fKyaxT0&$where=date_extract_y(cmplnt_fr_dt) in (2018)"
limit_url = "&$limit=50000"
i=1


#Connect to MongoDB
mdb_client = MongoClient("mongodb://localhost:27017/")
#Create Database
project_db = mdb_client["NewYorkData"]
#Create Collection
complaint_col = project_db["ComplaintsCollection"]

#If records already present
existing_records_count = complaint_col.find().count()
if (existing_records_count > 0):
    complaint_col.remove()

while i <= number_of_pages :
    page_url = url+limit_url+"&$offset="+str(i)
    page_response = requests.get(page_url)
    complaint_data = json.loads(page_response.text)
    #Count the number of records fetched from API
    fetched_records_count = len(complaint_data)
    #print(fetched_records_count)
    #Insert data to the ComplaintsCollection
    inserted_records = complaint_col.insert_many(complaint_data)
    inserted_records_count = len(inserted_records.inserted_ids)
    print(fetched_records_count == inserted_records_count)
    i=i+1

print(i)
url_last_page = url+"&$limit="+str(last_page_record_count)+"&$offset="+str(last_page_number)
last_page_repsonse = requests.get(url_last_page)
complaint_data_last_page = json.loads(last_page_repsonse.text)
#Count the number of records fetched from API
fetched_records_count = len(complaint_data_last_page)
print(fetched_records_count)
#Insert data to the ComplaintsCollection
inserted_records = complaint_col.insert_many(complaint_data_last_page)
inserted_records_count = len(inserted_records.inserted_ids)
print(inserted_records_count)

#Check count of inserted records
inserted_count = complaint_col.find().count()
print(inserted_count)

452997
2997


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


True
True
True
True
True
True
True
True
True
10
2997
2997
452997


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
